In [ ]:
# Importing the libraries
import numpy as np
import minterpy as mp
import minterpy_levelsets as ls

## Test 1 : Unit sphere

### Generate random points lying on unit sphere

In [ ]:
num_points = 11
sphere_points = ls.points_on_ellipsoid(num_points, radius_x=1.0, radius_y=1.0, radius_z=1.0, random_seed=42)

### Find the polynomial whose zero-contour passes through the set of points

In [ ]:
## Given the coordinates of the points, we compute the polynomial for a level function 
## whose zero passes through the given points

poly = ls.LevelsetPoly(sphere_points, method='BK')

In [ ]:
# Output the polynomial as VTK Rectilinear grid.
# poly.output_VTR()

### Compute gradients (unnormalized) on the points

In [ ]:
gradients = poly.compute_gradients_at(sphere_points)

In [ ]:
# unit normals
unit_normals = sphere_points / np.linalg.norm(sphere_points,2, axis=1)[:, None]

In [ ]:
# The projection of the normal vector along tangential direction
np.max(np.linalg.norm(gradients - np.sum(gradients*unit_normals, axis=1)[:,None]*unit_normals, np.inf, axis=1))

### Compute the mean and Gauss curvatures

In [ ]:
mc, gc = poly.compute_curvatures_at(sphere_points)

print(f"Mean curvatures : {mc}")
print(f"Gauss curvatures : {gc}")

## Test 2: Ellipsoid

In [ ]:
# Get coordinates of a set of points on a unit sphere
num_points = 100
p_a = 0.6
p_b = 0.8
p_c = 1.0
ellipsoid_points = ls.points_on_ellipsoid(num_points, radius_x=p_a, radius_y=p_b, radius_z=p_c)

In [ ]:
# Find the polynomial whose zero-contour passes through the set of points
poly = ls.LevelsetPoly(ellipsoid_points, method='LB')

### Error in gradient computation

In [ ]:
# Accuracy of normal vectors
exact_grad = np.zeros((num_points,3))

for i in range(num_points):
    exact_grad[:,0] = 2.0 * ellipsoid_points[:,0] / (p_a * p_a)
    exact_grad[:,1] = 2.0 * ellipsoid_points[:,1] / (p_b * p_b)
    exact_grad[:,2] = 2.0 * ellipsoid_points[:,2] / (p_c * p_c)

# Compute gradients (unnormalized) on the points
gradients = poly.compute_gradients_at(ellipsoid_points)    

gradients = gradients / np.linalg.norm(gradients,2, axis=1)[:, None]

exact_grad = exact_grad / np.linalg.norm(exact_grad,2,axis=1)[:,None]
    
max_grad_error = np.max(np.concatenate((np.abs(gradients[:,0] - exact_grad[:,0]), np.abs(gradients[:,1] - exact_grad[:,1]), np.abs(gradients[:,2] - exact_grad[:,2]))))
print(f"Error in gradient computation : {max_grad_error}")

### Error in curvature estimation

The ellipsoid is represented by the polynomial 

$ \frac{x^2}{a^2} + \frac{y^2}{b^2} + \frac{z^2}{c^2} - 1 = 0$

Gauss curvature is given by

$ K = \frac{1}{(abc)^2 \left( \frac{x^2}{a^4} + \frac{y^2}{b^4} + \frac{z^2}{c^4} \right)^2} $

and Mean curvature is given by

$ H = \frac{|x^2 + y^2 + z^2 - a^2 - b^2 - c^2|}{2(abc)^2 \left( \frac{x^2}{a^4} + \frac{y^2}{b^4} + \frac{z^2}{c^4} \right)^{3/2} } $


(from : 10.11648/j.larp.20170202.13 )

In [ ]:
mean_curvature, gauss_curvature = poly.compute_curvatures_at(ellipsoid_points)

In [ ]:
exact_mc = np.zeros(num_points)
exact_gc = np.zeros(num_points)

for i in range(num_points):
    x, y, z = ellipsoid_points[i,:]
    a = p_a
    b = p_b
    c = p_c
    #exact_mc[i] = -np.abs(x**2 + y**2 + z**2 - p_a**2 - p_b**2 - p_c**2)/(2*((p_a*p_b*p_c)**2)*(x**2/(p_a**4) + y**2/(p_b**4) + z**2/(p_c**4))**1.5)
    exact_mc[i] = -(a**2*c**2*(a**2 + c**2)*y**2 + b**4*(c**2*x**2 + a**2*z**2) + b**2*(c**4*x**2 + a**4*z**2))/(2.*a**4*b**4*c**4*(x**2/a**4 + y**2/b**4 + z**2/c**4)**1.5)
    exact_gc[i] = 1.0 / (((p_a*p_b*p_c)**2) * (x**2/(p_a**4) + y**2/(p_b**4) + z**2/(p_c**4))**2)

In [ ]:
# Note : We compare only the absolute value of mena curvature as the sign of the curvature depends on convention

max_mc_error = np.max(np.abs(mean_curvature) - np.abs(exact_mc))
max_gc_error = np.max(np.abs(gauss_curvature - exact_gc))
print(f"Error in mean curvature : {max_mc_error}")
print(f"Error in gauss curvature : {max_gc_error}")

## Test 3 : Torus

In [ ]:
num_points = 500

# Parameters for the surface
p_c = 0.5
p_a = 0.1

torus_points = ls.points_on_torus(num_points, p_c, p_a)

In [ ]:
torus_poly = ls.LevelsetPoly(torus_points, method='BK', tol=1e-7, verbose=True)

In [ ]:
def get_exact_curvatures_torus(pointcloud, par_c, par_a):
    N_points = pointcloud.shape[0]
    exact_mean_curvatures = np.zeros(N_points)
    exact_gauss_curvatures = np.zeros(N_points)
    
    for p in range(N_points):
        x, y, z = pointcloud[p, :]
        
        t1 = np.sqrt(x*x + y*y)
        t2 = (t1 - par_c) / par_a
        
        exact_gauss_curvatures[p] = t2/(par_a*t1)
        exact_mean_curvatures[p] = (par_c - 2*t1) / (2*par_a*t1)
        
    return exact_mean_curvatures, exact_gauss_curvatures

In [ ]:
exact_mc, exact_gc = get_exact_curvatures_torus(torus_points, p_c, p_a)

In [ ]:
mean_curvature, gauss_curvature = torus_poly.compute_curvatures_at(torus_points)

In [ ]:
max_mc_error = np.max(np.abs(mean_curvature) - np.abs(exact_mc))
max_gc_error = np.max(np.abs(gauss_curvature - exact_gc))

print(f"L_inf error in Mean Curvature is {max_mc_error}")
print(f"L_inf erro rin Gauss Curvature is {max_gc_error}")